# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
!pip install hvplot cartopy geoviews

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,16.61,72,4,0.45,US,1725838404
1,1,edinburgh of the seven seas,-37.0676,-12.3116,12.65,75,100,7.83,SH,1725838405
2,2,georgetown,5.4112,100.3354,25.96,87,20,1.03,MY,1725838408
3,3,hermanus,-34.4187,19.2345,10.20,71,46,4.82,ZA,1725838306
4,4,deqen,29.9618,90.7188,7.42,86,79,1.68,CN,1725838308
...,...,...,...,...,...,...,...,...,...,...
584,584,taguatinga,-12.3857,-46.5711,27.77,21,0,4.05,BR,1725839183
585,585,yaren,-0.5472,166.9160,28.22,77,83,5.43,NR,1725839184
586,586,jingdezhen,29.2947,117.2079,28.68,67,0,1.45,CN,1725839185
587,587,iklehra,23.1500,76.3833,22.73,95,29,4.62,IN,1725839186


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [5]:
### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition
# Narrow down cities that fit criteria and drop any results with null values
city_data_filter_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) 
& (city_data_df['Max Temp'] < 27) 
& (city_data_df['Wind Speed'] < 4.5) 
& (city_data_df['Cloudiness'] == 0)]
                                        
# Drop any rows with null values
city_data_filter_df = city_data_filter_df.dropna()

# Display sample data
city_data_filter_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
68,68,douglas,41.3000,-96.1670,25.49,45,0,3.60,US,1725838488
177,177,babolsar,36.7025,52.6576,23.54,71,0,1.42,IR,1725838618
262,262,tamanrasset,22.7850,5.5228,25.95,47,0,1.54,DZ,1725838719
278,278,avia terai,-26.6853,-60.7292,26.96,26,0,2.17,AR,1725838739
283,283,pacific grove,36.6177,-121.9166,22.99,86,0,3.58,US,1725838745
287,287,al jawf,29.5000,38.7500,26.73,33,0,2.92,SA,1725838749
303,303,santa isabel,-23.3156,-46.2214,23.61,43,0,1.25,BR,1725838769
342,342,arinos,-15.9169,-46.1056,23.08,26,0,0.68,BR,1725838815
449,449,tsiombe,-25.3000,45.4833,23.10,51,0,1.68,MG,1725838947
450,450,moires,35.0532,24.8744,23.95,52,0,1.79,GR,1725838948


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filter_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
68,douglas,US,41.3000,-96.1670,45,
177,babolsar,IR,36.7025,52.6576,71,
262,tamanrasset,DZ,22.7850,5.5228,47,
278,avia terai,AR,-26.6853,-60.7292,26,
283,pacific grove,US,36.6177,-121.9166,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories':'accommodation.hotel', 'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    # # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # # Convert the API response to JSON format
    name_address = name_address.json()
    
    # # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
         hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        #If no hotel is found, set the hotel name as "No hotel found".
         hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
douglas - nearest hotel: Holiday Inn Express
babolsar - nearest hotel: هتل رزآیند
tamanrasset - nearest hotel: فندق الأمان
avia terai - nearest hotel: Las Curiosas Turismo Rural
pacific grove - nearest hotel: Pacific Grove Inn
al jawf - nearest hotel: No hotel found
santa isabel - nearest hotel: No hotel found
arinos - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
moires - nearest hotel: Hotel Olympic
najran - nearest hotel: No hotel found
kencong - nearest hotel: Grand Gumuk Mas Hotel
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham


,City,Country,Lat,Lng,Humidity,Hotel Name
68,douglas,US,41.3000,-96.1670,45,Holiday Inn Express
177,babolsar,IR,36.7025,52.6576,71,هتل رزآیند
262,tamanrasset,DZ,22.7850,5.5228,47,فندق الأمان
278,avia terai,AR,-26.6853,-60.7292,26,Las Curiosas Turismo Rural
283,pacific grove,US,36.6177,-121.9166,86,Pacific Grove Inn
287,al jawf,SA,29.5000,38.7500,33,No hotel found
303,santa isabel,BR,-23.3156,-46.2214,43,No hotel found
342,arinos,BR,-15.9169,-46.1056,26,No hotel found
449,tsiombe,MG,-25.3000,45.4833,51,No hotel found
450,moires,GR,35.0532,24.8744,52,Hotel Olympic


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_data_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name","Country"]
)

# Display the map
hotel_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)